In [3]:
import geopandas as gpd
import pandas as pd
import unidecode
#Importa os dados
dado = pd.read_excel('MICRODADOS_ESTUPRO_JAN_2015_A_NOV_2022.xlsx')
data = gpd.read_file("PE_Municipios_2021.shp")

In [2]:
display(dado)

,MUNICÍPIO DO FATO,REGIAO GEOGRÁFICA,NATUREZA,DATA DO FATO,ANO,SEXO,IDADE SENASP,TOTAL DE ENVOLVIDOS
0,AGUAS BELAS,AGRESTE,ESTUPRO DE VULNERÁVEL POR VIOLÊNCIA DOMÉSTICA/...,2015-01-01,2015,FEMININO,2) 12-17,1
1,ARCOVERDE,SERTÃO,ESTUPRO DE VULNERÁVEL,2015-01-01,2015,FEMININO,2) 12-17,1
2,BEZERROS,AGRESTE,ESTUPRO DE VULNERÁVEL,2015-01-01,2015,MASCULINO,2) 12-17,1
3,BONITO,AGRESTE,ESTUPRO,2015-01-01,2015,FEMININO,1) 00-11,1
4,BREJAO,AGRESTE,ESTUPRO DE VULNERÁVEL,2015-01-01,2015,FEMININO,1) 00-11,1
...,...,...,...,...,...,...,...,...
19319,OLINDA,REGIÃO METROPOLITANA,ESTUPRO DE VULNERÁVEL POR VIOLÊNCIA DOMÉSTICA/...,2022-11-16,2022,FEMININO,1) 00-11,1
19320,OLINDA,REGIÃO METROPOLITANA,ESTUPRO,2022-11-25,2022,FEMININO,2) 12-17,1
19321,PALMARES,ZONA DA MATA,ESTUPRO DE VULNERÁVEL POR VIOLÊNCIA DOMÉSTICA/...,2022-09-15,2022,FEMININO,1) 00-11,1
19322,ARCOVERDE,SERTÃO,ESTUPRO DE VULNERÁVEL,2022-11-03,2022,FEMININO,1) 00-11,1


In [82]:
#torna a coluna de nomes sem acento e tudo minusculo
for c in range(0,len(data.index)):
    data.iat[c,1]=unidecode.unidecode(data.iat[c,1].lower())
coords = {}
x = 0
#For na tabela do crime
for c in dado['MUNICÍPIO DO FATO']:
    #tratamento de cidades específicas e pega os dados
    if(c=='ITAMARACA'):
        dado_mun = data[data['NM_MUN']=='ilha de itamaraca']
    elif(c=='SAO CAETANO'):
        dado_mun = data[data['NM_MUN']=='sao caitano']
    elif(c=='BELEM DE SAO FRANCISCO'):
        dado_mun = data[data['NM_MUN']=='belem do sao francisco']
    else:
        dado_mun = data[data['NM_MUN']==c.lower()]
    #salva apenas o nome da cidade
    geo = str(dado_mun['NM_MUN']).split('    ')
    geo = geo[1].replace('Name: NM_MUN, dtype: object',"")
    geo = geo.replace('\n','')
    #se a cidade ainda não foi cadastrada adiciona ela no dicionario
    if(coords.get(geo,'X')=='X'):
        coords[geo]=1
    #caso ela já tenha sido adicionada, soma mais um no número de casos
    else:
        valor = coords[geo]
        coords.update({geo:valor+1})
        
    

print(coords)

{'aguas belas': 72, 'arcoverde': 142, 'bezerros': 119, 'bonito': 98, 'brejao': 35, 'caruaru': 751, 'cha de alegria': 26, 'cumaru': 37, 'cupira': 77, 'flores': 25, 'jaboatao dos guararapes': 1278, 'moreno': 117, 'olinda': 971, 'paudalho': 136, 'paulista': 860, 'pedra': 51, 'petrolina': 799, 'recife': 3259, 'riacho das almas': 41, 'saloa': 28, 'serra talhada': 132, 'vitoria de santo antao': 278, 'moreilandia': 23, 'quipapa': 40, 'camaragibe': 341, 'cha grande': 36, 'goiana': 168, 'ilha de itamaraca': 87, 'afogados da ingazeira': 96, 'agua preta': 58, 'escada': 152, 'gameleira': 31, 'ipojuca': 322, 'pombos': 74, 'rio formoso': 43, 'sao lourenco da mata': 288, 'itambe': 71, 'sao benedito do sul': 19, 'sao bento do una': 126, 'sirinhaem': 71, 'ibimirim': 46, 'ribeirao': 70, 'timbauba': 71, 'afranio': 30, 'feira nova': 33, 'mirandiba': 32, 'petrolandia': 80, 'tuparetama': 16, 'belo jardim': 177, 'palmeirina': 11, 'pesqueira': 102, 'cabo de santo agostinho': 564, 'lagoa grande': 57, 'itapissu

In [84]:
print(coords['recife'])

3259


In [76]:
dado_mun = data[data['NM_MUN']=='aguas belas']
geo = str(dado_mun['NM_MUN']).split('    ')
geo = geo[1].replace('Name: NM_MUN, dtype: object',"")
geo = geo.replace('\n','')
print(geo+"ponei")

aguas belasponei


In [109]:
import folium
from folium.plugins import FastMarkerCluster
#gera o mapa com um foco em uma localidade
m = folium.Map(location=[-7.6437599,-36.8682613],zoom_start=7.4,tiles='CartoDB positron')
#faz um for nas linhas
for _, r in data.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    #converte o shapefile poligono em geojson
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    #trata a quantidade de casos em uma cidade
    if(coords[r['NM_MUN']]<50):
        #adiciona o poligono no geojson de uma determinada cor
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'white'})
    elif(coords[r['NM_MUN']]<250):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'yellow'})
    elif(coords[r['NM_MUN']]<1000):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'red'})
    else:
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'purple'})
    #salva a quantidade de casos
    info = str(coords[r['NM_MUN']])
    #adiciona um popup ao geojson
    folium.Popup(r['NM_MUN']+": "+info).add_to(geo_j)
    geo_j.add_to(m)
#exibe o geojson
m


In [132]:
#adiciona a tabela de populacao das cidades por estado
col = ["muni",	"Código [-]",	"Gentílico [-]",	"Prefeito [2021]",	"Área Territorial - km² [2021]",	"populacao"	,"Densidade demográfica - hab/km² [2010]",	"Escolarização <span>6 a 14 anos</span> - % [2010]",	"IDHM <span>Índice de desenvolvimento humano municipal</span> [2010]",	"Mortalidade infantil - óbitos por mil nascidos vivos [2020]","	Receitas realizadas - R$ (×1000) [2017]",	"Despesas empenhadas - R$ (×1000) [2017]","	PIB per capita - R$ [2020]"]
muns_pop = pd.read_excel('pe_mun_dados.xlsx',names=col)
muns_pop = muns_pop.drop(1)
muns_pop = muns_pop.dropna()


In [135]:
for c in range(0,len(muns_pop.index)):
    muns_pop.iat[c,0]=unidecode.unidecode(muns_pop.iat[c,0].lower())



1661017


In [146]:


m = folium.Map(location=[-7.6437599,-36.8682613],zoom_start=7.4,tiles='CartoDB positron')
for _, r in data.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    info = coords[r['NM_MUN']]/8
    mun_unic = muns_pop[muns_pop['muni']==r['NM_MUN']]
    pop = str(mun_unic['populacao']).split('    ')
    pop = pop[1].replace('Name: populacao, dtype: object',"")
    pop = pop.replace('\n','')
    porcen = info/(int(pop)/100000)
    if(porcen<10):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'white'})
    elif(porcen<15):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'yellow'})
    elif(porcen<20):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'red'})
    elif(porcen<25):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'purple'})
    else:
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'black'})
    folium.Popup(r['NM_MUN']+"\n"+"População: "+pop+"\nCasos: "+str(info)+"\n Por 100K/Hab: "+str(porcen)).add_to(geo_j)
    geo_j.add_to(m)
m